

This is where the introduction to the analysis and the title goes

## Importing the required packages

Numpy, pandas and sklear are all needed for this analysis. These are imported below.

In [ ]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from sklearn.linear_model import Lasso

## Extracting the data for this analysis

Sixteen predictors were considered for this analysis, of which 12 were ultimately used. For each of the variables, the latest year with the most complete data that was available were selected for the analysis. The details of the variables included, as well as the outcome (life expectancy), are listed in the table below. [Smoking rates](http://apps.who.int/gho/data/node.main.1250?lang=en), [rates of insufficient physical activity](http://apps.who.int/gho/data/node.main.A893?lang=en), and [malaria](http://apps.who.int/gho/data/view.main.14111?lang=en) and [HIV](http://apps.who.int/gho/data/node.main.620?lang=en) incidence were also considered for this model, but were found to have high rates of missing data and were therefore excluded.

These analyses were performed on data extracted on the 24th July 2016.

**Description of data used for each variable**

| Variable | Year | Measure used | Gender sample | Source URL |
| -------- | ---- | ------------ | ------------- | ---------- |
| Life expectancy | 2015 | Life expectancy at birth | Both sexes | http://apps.who.int/gho/data/node.main.688?lang=en|
| Alcohol | 2010 | Per capita consumption (in litres) | All beverage types | http://apps.who.int/gho/data/node.main.A1026?lang=en|
| Overweight | 2014 | Age-standarised rate (BMI >= 25) | Both sexes | http://apps.who.int/gho/data/node.main.A897A?lang=en|
| Cholesterol | 2008 | Age-standardised rate | Both sexes | http://apps.who.int/gho/data/node.main.A884?lang=en |
| Blood sugar | 2014 | Crude rate | Both sexes | http://apps.who.int/gho/data/node.main.A869?lang=en |
| Improved water | 2015 | % using improved water sources | Total | http://apps.who.int/gho/data/node.main.167?lang=en | 
| Improved sanitation | 2015 | % using improved sanitation | Total | http://apps.who.int/gho/data/node.main.167?lang=en |
| Maternal deaths | 2015 | Maternal mortality ratio | N/A  | http://apps.who.int/gho/data/node.main.MATMORT?lang=en |
| UV radiation | 2004 | Exposure to solar UV | N/A | http://apps.who.int/gho/data/node.main.164?lang=en |
| Homicide | 2012 | Rate of homicides | N/A | http://apps.who.int/gho/data/node.main.VIOLENCEHOMICIDE?lang=en |
| Traffic deaths | 2013 | Rate of road traffic deaths | N/A | http://apps.who.int/gho/data/node.main.A997?lang=en |
| Tuberculosis | 2014 | Incidence of tuberculosis | All cases | http://apps.who.int/gho/data/view.main.57040ALL?lang=en |
| Suicide | 2012 | Age-standardised rate | Both sexes | http://apps.who.int/gho/data/node.main.MHSUICIDE?lang=en |

The variables for this analysis are extracted directly from the website below:

In [ ]:
def dataImport(dataurl):
	url = dataurl
	return pd.read_csv(url)

life = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/WHOSIS_000001,WHOSIS_000015&profile=crosstable&filter=COUNTRY:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO;SEX")
alcohol = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/SA_0000001400&profile=crosstable&filter=COUNTRY:*;YEAR:2015;YEAR:2014;YEAR:2013;YEAR:2012;YEAR:2011;YEAR:2010;YEAR:2009;YEAR:2008;YEAR:2007;YEAR:2006;YEAR:2005;YEAR:2004;YEAR:2003;YEAR:2002;YEAR:2001;YEAR:2000&x-sideaxis=COUNTRY;DATASOURCE;ALCOHOLTYPE&x-topaxis=GHO;YEAR")
oweight = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/NCD_BMI_25A&profile=crosstable&filter=AGEGROUP:*;COUNTRY:*;SEX:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR;AGEGROUP;SEX")
chol = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/CHOL_01,CHOL_02&profile=crosstable&filter=AGEGROUP:*;COUNTRY:*;SEX:*&x-sideaxis=COUNTRY;YEAR;AGEGROUP&x-topaxis=GHO;SEX")
bsugar = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/NCD_GLUC_03,NCD_GLUC_04&profile=crosstable&filter=AGEGROUP:*;COUNTRY:*;SEX:*&x-sideaxis=COUNTRY;YEAR;AGEGROUP&x-topaxis=GHO;SEX")
sanitation = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/WHS5_122,WHS5_158&profile=crosstable&filter=COUNTRY:*;RESIDENCEAREATYPE:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO;RESIDENCEAREATYPE")
maternal = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/MDG_0000000026,MORT_MATERNALNUM&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO")
uvrad = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/UV_1&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR")
homicides = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/VIOLENCE_HOMICIDENUM,VIOLENCE_HOMICIDERATE&profile=crosstable&filter=COUNTRY:*;AGEGROUP:-;SEX:-&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR")
traffdeath = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/RS_196,RS_198&profile=crosstable&filter=COUNTRY:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR")
tb = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/MDG_0000000020,TB_e_inc_num,TB_e_inc_tbhiv_100k,TB_e_inc_tbhiv_num&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO")
suicide = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/MH_12&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR;SEX")

## Cleaning the data

Each of the variables have been imported in separate DataFrames that need to be cleaned. These need to be cleaned so that only the relevant years and variables are remaining. In addition